<a href="https://colab.research.google.com/github/Hack-it-Rafi/Machine_Learning_Classwork/blob/main/hw4/logistic-regression-lab-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [12]:
# Load in additional functions
import pandas as pd

# Use three digits past the decimal point, and don't use scientific notation.
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None) # Display all columns

# The R 'theme_set' and 'theme_update' commands are specific to ggplot2 in R
# and do not have direct equivalents for data loading and sampling in Python.
# If plotting is desired later, libraries like matplotlib or seaborn would be used.

# Read in the data
spam = pd.read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
print(spam.sample(10))

      make  address   all    3d   our  over  remove  internet  order  mail  \
3861 0.190    0.000 0.590 0.000 0.000 0.000   0.000     0.000  0.000 0.390   
4428 0.000    0.000 1.130 0.000 0.370 0.000   0.000     0.000  0.000 0.000   
1881 0.000    0.000 0.000 0.000 0.000 0.000   0.000     0.000  0.000 0.000   
4414 0.000    0.000 1.010 0.000 0.000 1.010   0.000     0.000  0.000 1.010   
2601 0.000    0.000 0.000 0.000 0.000 0.000   0.000     0.000  0.000 0.000   
1169 0.000    0.890 1.150 0.000 0.120 0.000   0.000     0.120  0.250 0.120   
1687 0.000    0.000 0.000 0.000 0.000 0.000   0.000     0.000  0.000 0.000   
4367 0.000    0.000 0.000 0.000 0.000 0.000   0.000     0.000  0.000 0.000   
4001 0.000    0.000 0.000 0.000 0.000 0.000   0.000     0.000  0.000 0.000   
3262 0.220    0.000 0.070 0.000 0.070 0.070   0.000     0.140  0.000 0.360   

      receive  will  people  report  addresses  free  business  email   you  \
3861    0.000 0.000   0.000   0.000      0.000 0.000     0.000

## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [13]:
# 1. How many emails are in the database?
total_emails = len(spam)
print(f"Total emails: {total_emails}")

spam_count = spam['is_spam'].sum()
fraction_spam = spam_count / total_emails
print(f"Number of spam emails: {spam_count}")
print(f"Fraction spam: {fraction_spam:.4f}")

max_money_percentage = spam['money'].max()
email_with_highest = spam['money'].idxmax()

print(f"Highest 'money' percentage: {max_money_percentage}%")
print(f"Found in email index (0-based): {email_with_highest}")
print(f"Found in email index (1-based): {email_with_highest + 1}")



Total emails: 4601
Number of spam emails: 1813
Fraction spam: 0.3940
Highest 'money' percentage: 12.5%
Found in email index (0-based): 545
Found in email index (1-based): 546


## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [14]:
# Your code here!
import statsmodels.api as sm


X = spam[['char_dollar', 'credit', 'money', 're']]
y = spam['is_spam']

X = sm.add_constant(X)

lpm = sm.OLS(y, X).fit()

print(lpm.summary())

print("\n" + "="*60)
print("LINEAR PROBABILITY MODEL RESULTS")
print("="*60)
print(lpm.params)
print("\nInterpretation:")
print(f"Intercept:                  {lpm.params['const']:.4f}")
print(f"   → When all predictors are 0, predicted probability of spam = {lpm.params['const']:.4f} ({lpm.params['const']*100:.2f}%)")

print(f"\nchar_dollar coefficient:    {lpm.params['char_dollar']:.6f}")

print("\nFull coefficient table:")
print(lpm.summary2().tables[1][['Coef.', 'Std.Err.', 'P>|t|', '[0.025', '0.975]']])


                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        17:25:21   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Interpretation:
- Intercept = 0.2970 → If an email has 0% '$', 0% 'credit', 0% 'money', and 0% 're',
  the predicted probability it is spam is about 29.7%.

- char_dollar coefficient = 0.0865 → Each additional 1% of characters that are '$'
  increases the predicted probability of being spam by 8.65 percentage points.

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [15]:
# Your code here!
spam['pred_prob'] = lpm.predict(X)

min_prob = spam['pred_prob'].min()
max_prob = spam['pred_prob'].max()

print(f"Smallest predicted probability: {min_prob:.6f}")
print(f"Largest predicted probability:  {max_prob:.6f}")

print("\nEmails with predicted probability < 0:")
print(spam[spam['pred_prob'] < 0][['pred_prob', 'char_dollar', 'credit', 'money', 're', 'is_spam']])

print("\nEmails with predicted probability > 1:")
print(spam[spam['pred_prob'] > 1][['pred_prob', 'char_dollar', 'credit', 'money', 're', 'is_spam']])

print("\nIssues with LPM predictions:")
print(f"- {len(spam[spam['pred_prob'] < 0])} emails have predicted probability < 0")
print(f"- {len(spam[spam['pred_prob'] > 1])} emails have predicted probability > 1")
print("- Linear probability model can predict values outside [0,1] range")
print("- This is a known limitation of LPM when used for binary outcomes")


Smallest predicted probability: -0.812540
Largest predicted probability:  3.849409

Emails with predicted probability < 0:
      pred_prob  char_dollar  credit  money     re  is_spam
1977     -0.011        0.000   0.000  0.000  6.450        0
1982     -0.077        0.000   0.000  0.000  7.690        0
2105     -0.112        0.000   0.000  0.000  8.330        0
2386     -0.112        0.000   0.000  0.000  8.330        0
2541     -0.077        0.000   0.000  0.000  7.690        0
2621     -0.022        0.000   0.000  0.000  6.660        0
2695     -0.558        0.000   0.000  0.000 16.660        0
2698     -0.048        0.000   0.000  0.000  7.140        0
2704     -0.077        0.000   0.000  0.000  7.690        0
2720     -0.152        0.000   0.000  0.000  9.090        0
2778     -0.048        0.000   0.000  0.000  7.140        0
2847     -0.048        0.000   0.000  0.000  7.140        0
2904     -0.000        0.000   0.000  0.000  6.250        0
3040     -0.048        0.000   0.000 

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [16]:
# Your code here!
def prob_to_odds(p):
    """Convert probability to odds (as a ratio, e.g. 2.0 means 2:1)"""
    if p <= 0 or p >= 1:
        raise ValueError("Probability must be between 0 and 1 (exclusive)")
    return p / (1 - p)


def odds_to_prob(odds):
    """Convert odds (e.g. 2.0 for 2:1) to probability"""
    if odds < 0:
        raise ValueError("Odds cannot be negative")
    return odds / (odds + 1)


print("2:1 odds → probability:")
print(odds_to_prob(2))

print("\nprobability 2/3 → odds:")
print(prob_to_odds(2/3))

print("\nRound-trip tests:")
print(prob_to_odds(odds_to_prob(2)) == 2)
print(odds_to_prob(prob_to_odds(2/3)) == 2/3)


p_original = 0.60
odds_original = prob_to_odds(p_original)
odds_doubled = 2 * odds_original
p_new = odds_to_prob(odds_doubled)

print("\nIf original probability is 60%:")
print(f"Original odds: {odds_original:.3f}:1")
print(f"Doubled odds:  {odds_doubled:.3f}:1")
print(f"New probability: {p_new:.1%}")


2:1 odds → probability:
0.6666666666666666

probability 2/3 → odds:
1.9999999999999998

Round-trip tests:
False
True

If original probability is 60%:
Original odds: 1.500:1
Doubled odds:  3.000:1
New probability: 75.0%


## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [17]:
X = spam[['char_dollar', 'credit', 'money', 're']]
X = sm.add_constant(X)

y = spam['is_spam']

model = sm.GLM(y, X, family=sm.families.Binomial())
result = model.fit()

print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                            GLM   Df Residuals:                     4596
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2213.9
Date:                Tue, 18 Nov 2025   Deviance:                       4427.8
Time:                        17:25:21   Pearson chi2:                 1.00e+10
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3152
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -1.0666      0.043    -24.680      

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [18]:
import numpy as np

X = spam[['char_dollar', 'credit', 'money', 're']]
X = sm.add_constant(X)
y = spam['is_spam']

model = sm.GLM(y, X, family=sm.families.Binomial())
result = model.fit()

coef = result.params
print("Logistic Regression Coefficients:")
print(coef.round(4))

intercept = coef['const']
money_coef = coef['money']

print("\n" + "="*70)
print("INTERPRETATION OF LOGISTIC REGRESSION COEFFICIENTS")
print("="*70)

print("\n1. On the LOG-ODDS scale:")
print(f"   Intercept: {intercept:.4f}")
print(f"     → When all predictors = 0, log-odds of spam = {intercept:.4f}")
print(f"   money:     {money_coef:.4f}")
print(f"     → A 1 percentage point increase in 'money' words")
print(f"       increases log-odds of spam by {money_coef:.4f}")

print("\n2. On the ODDS scale (exponentiated):")
print(f"   exp(Intercept) = {np.exp(intercept):.4f}")
print(f"     → Baseline odds of spam (when all predictors=0): 1 to {np.exp(intercept):.2f}")
print(f"   exp(money)     = {np.exp(money_coef):.4f}")
print(f"     → Each 1% increase in 'money' multiplies odds of spam by {np.exp(money_coef):.3f}")

print("\n3. On the PROBABILITY scale:")

baseline_odds = np.exp(intercept)
baseline_prob = baseline_odds / (1 + baseline_odds)
money_odds_multiplier = np.exp(money_coef)
prob_increase_approx = money_coef / 4

print(f"   Baseline probability (all predictors=0): {baseline_prob:.1%}")
print(f"   Approximate marginal effect of 1% more 'money' words:")
print(f"     → Increases spam probability by ≈ {prob_increase_approx:.4f} ({prob_increase_approx*100:.2f} percentage points)")
print(f"     (using the 'divide by 4' rule: β/4)")

odds_after = baseline_odds * money_odds_multiplier
prob_after = odds_after / (1 + odds_after)
exact_increase = prob_after - baseline_prob

print(f"   Exact increase (when other variables = 0): +{exact_increase*100:.2f} percentage points")


Logistic Regression Coefficients:
const         -1.067
char_dollar   11.818
credit         2.312
money          1.993
re            -0.775
dtype: float64

INTERPRETATION OF LOGISTIC REGRESSION COEFFICIENTS

1. On the LOG-ODDS scale:
   Intercept: -1.0666
     → When all predictors = 0, log-odds of spam = -1.0666
   money:     1.9933
     → A 1 percentage point increase in 'money' words
       increases log-odds of spam by 1.9933

2. On the ODDS scale (exponentiated):
   exp(Intercept) = 0.3442
     → Baseline odds of spam (when all predictors=0): 1 to 0.34
   exp(money)     = 7.3396
     → Each 1% increase in 'money' multiplies odds of spam by 7.340

3. On the PROBABILITY scale:
   Baseline probability (all predictors=0): 25.6%
   Approximate marginal effect of 1% more 'money' words:
     → Increases spam probability by ≈ 0.4983 (49.83 percentage points)
     (using the 'divide by 4' rule: β/4)
   Exact increase (when other variables = 0): +46.04 percentage points
